# Task to create the taxi raw records (bronze)

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
bronze_tbl = f"{catalog}.{schema}.gha_ss_taxi_raw_records_bronze"

In [0]:
# location to store checkpoint files
spark.sql(f"create volume if not exists {catalog}.{schema}.checkpoint_files")
checkpoint_path = f"/Volumes/{catalog}/{schema}/checkpoint_files/top_n/gha_ss_taxi_raw_records_bronze_checkpoint"

In [0]:
from pyspark.sql.functions import *

streamdf = (spark.readStream.table("samples.nyctaxi.trips")
 .filter(col("trip_distance") > 0.0)
 .writeStream
 .outputMode("append")
 .option("checkpointLocation", checkpoint_path)
 .trigger(availableNow=True)
 .toTable(bronze_tbl)
)

streamdf.awaitTermination()